<a href="https://colab.research.google.com/github/JK-the-Ko/Thermo_Fluid_Dynamics_Experiment/blob/main/2021_2/%EC%97%B4%EC%9C%A0%EC%B2%B4%EA%B3%B5%ED%95%99%EC%8B%A4%ED%97%98_Week_13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Check GPU Connection**

In [ ]:
!nvidia-smi

**Import Library**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras

**Load Fashion CIFAR10 Dataset**

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

In [ ]:
num_classes = 10
input_shape = (32, 32, 3)

**Check Number of Train Data**

In [ ]:
print(x_train.shape)
print(y_train.shape)

**Check Number of Test Data**

In [ ]:
print(x_test.shape)
print(y_test.shape)

**Show Sample**

In [ ]:
sample_image = x_train[0, :, :]

In [ ]:
sample_image.shape

In [ ]:
sample_image.dtype

In [ ]:
plt.rcParams["figure.figsize"] = (5, 5)
plt.imshow(sample_image)
plt.axis("off")
plt.show()

In [ ]:
sample_label = y_train[0]

In [ ]:
sample_label

**Data Processing**

* Min-Max Normalization

In [ ]:
x_train = x_train / 255
x_test = x_test / 255

In [ ]:
x_train.dtype

In [ ]:
print(np.max(x_train))
print(np.min(x_train))

* One-Hot Encoding

In [ ]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
y_train[0]

**Learning Rate Decay**

In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler

* Scheduler Function

In [ ]:
def scheduler(epoch, lr) :
  if epoch < 10 :
    return 1e-3
  
  elif 10 <= epoch < 20 :
    return 1e-4

  else :
    return 1e-5

In [ ]:
callback = LearningRateScheduler(scheduler)

* Make CNN Model

In [ ]:
from tensorflow.keras import layers

In [ ]:
model = keras.Sequential()

In [ ]:
model.add(layers.Conv2D(32, (3, 3), padding = "same", activation = "relu", input_shape = input_shape))
model.add(layers.BatchNormalization())

model.add(layers.Conv2D(32, (3, 3), padding = "same", activation = "relu"))
model.add(layers.BatchNormalization())

model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), padding = "same", activation = "relu"))
model.add(layers.BatchNormalization())

model.add(layers.Conv2D(64, (3, 3), padding = "same", activation = "relu"))
model.add(layers.BatchNormalization())

model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), padding = "same", activation = "relu"))
model.add(layers.BatchNormalization())

model.add(layers.Conv2D(128, (3, 3), padding = "same", activation = "relu"))
model.add(layers.BatchNormalization())

model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(128, activation = "relu"))
model.add(layers.Dense(128, activation = "relu"))
model.add(layers.Dense(num_classes, "softmax"))

* Compile Model

In [ ]:
from tensorflow.keras import optimizers

In [ ]:
model.compile(optimizer = optimizers.Adam(learning_rate = 1e-3), loss = "categorical_crossentropy", metrics = ["accuracy"])

* Train Model

In [ ]:
batch_size = 256
epochs = 30

In [ ]:
hist = model.fit(x_train, y_train, batch_size = batch_size, epochs = epochs, validation_split = 0.2, callbacks = [callback])

In [ ]:
plt.rcParams["figure.figsize"] = (10, 5)

In [ ]:
plt.plot(np.arange(0, epochs), hist.history["loss"], label = "Train Loss")
plt.plot(np.arange(0, epochs), hist.history["val_loss"], label = "Validation Loss")
plt.legend()
plt.show()

In [ ]:
plt.plot(np.arange(0, epochs), hist.history["accuracy"], label = "Train Accuracy")
plt.plot(np.arange(0, epochs), hist.history["val_accuracy"], label = "Validation Accuracy")
plt.legend()
plt.show()

**ResNet**

* ResNet Block

In [ ]:
class ResNet_Block(keras.Model) :
  def __init__(self, num_channels) :
    super(ResNet_Block, self).__init__()

    self.conv_1 = layers.Conv2D(num_channels, (3, 3), padding = "same")
    self.conv_2 = layers.Conv2D(num_channels, (3, 3), padding = "same")
    self.bn_1 = layers.BatchNormalization()
    self.bn_2 = layers.BatchNormalization()
    self.relu = layers.Activation("relu")

  def call(self, x) :
    out = self.conv_1(x)
    out = self.bn_1(out)
    out = self.relu(out)
    out = self.conv_2(out)
    out = self.bn_2(out)
    out = x + out
    out = self.relu(out)

    return out

* Make ResNet Model

In [ ]:
model = keras.Sequential()

In [ ]:
model.add(layers.Conv2D(32, (3, 3), padding = "same", activation = "relu", input_shape = input_shape))
model.add(ResNet_Block(32))
model.add(layers.MaxPool2D(2, 2))

model.add(layers.Conv2D(64, (3, 3), padding = "same", activation = "relu"))
model.add(ResNet_Block(64))
model.add(layers.MaxPool2D(2, 2))
model.add(ResNet_Block(64))
model.add(layers.MaxPool2D(2, 2))

model.add(layers.Conv2D(128, (3, 3), padding = "same", activation = "relu"))
model.add(ResNet_Block(128))

model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(128, activation = "relu"))
model.add(layers.Dense(128, activation = "relu"))
model.add(layers.Dense(num_classes, "softmax"))

In [ ]:
model.summary()

* Model Visualization

In [ ]:
from tensorflow.keras.utils import plot_model

In [ ]:
plot_model(model, "/content/model.png")

* Train ResNet

In [ ]:
model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])

In [ ]:
hist = model.fit(x_train, y_train, batch_size = batch_size, epochs = epochs, validation_split = 0.2, callbacks = [callback])

In [ ]:
plt.plot(np.arange(0, epochs), hist.history["loss"], label = "Train Loss")
plt.plot(np.arange(0, epochs), hist.history["val_loss"], label = "Validation Loss")
plt.legend()
plt.show()

In [ ]:
plt.plot(np.arange(0, epochs), hist.history["accuracy"], label = "Train Accuracy")
plt.plot(np.arange(0, epochs), hist.history["val_accuracy"], label = "Validation Accuracy")
plt.legend()
plt.show()

**Inception Module**

In [ ]:
class Inception_Module(keras.Model) :
  def __init__(self, num_channels) :
    super(Inception_Module, self).__init__()

    self.conv_1 = layers.Conv2D(num_channels // 4, (1, 1), activation = "relu")
    self.conv_2 = keras.Sequential([layers.Conv2D(num_channels // 4, (1, 1)),
                                    layers.Conv2D(num_channels // 4, (3, 3), padding = "same", activation = "relu")])
    self.conv_3 = keras.Sequential([layers.Conv2D(num_channels // 4, (1, 1)),
                                    layers.Conv2D(num_channels // 4, (5, 5), padding = "same", activation = "relu")])
    self.conv_4 = keras.Sequential([layers.MaxPool2D((3, 3), strides = 1, padding = "same"),
                                    layers.Conv2D(num_channels // 4, (1, 1), activation = "relu")])

  def call(self, x) :
    out = keras.layers.Concatenate()([self.conv_1(x), self.conv_2(x), self.conv_3(x), self.conv_4(x)])

    return out

* Make GoogLeNet Model

In [ ]:
model = keras.Sequential()

In [ ]:
model.add(layers.Conv2D(32, (3, 3), padding = "same", activation = "relu", input_shape = input_shape))
model.add(Inception_Module(32))
model.add(layers.MaxPool2D(2, 2))

model.add(layers.Conv2D(64, (3, 3), padding = "same", activation = "relu"))
model.add(Inception_Module(64))
model.add(layers.MaxPool2D(2, 2))
model.add(Inception_Module(64))
model.add(layers.MaxPool2D(2, 2))

model.add(layers.Conv2D(128, (3, 3), padding = "same", activation = "relu"))
model.add(Inception_Module(128))

model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(128, activation = "relu"))
model.add(layers.Dense(128, activation = "relu"))
model.add(layers.Dense(num_classes, "softmax"))

In [ ]:
model.summary()

* Model Visualization

In [ ]:
from tensorflow.keras.utils import plot_model

In [ ]:
plot_model(model, "/content/model.png")

* Train GoogLeNet

In [ ]:
model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])

In [ ]:
hist = model.fit(x_train, y_train, batch_size = batch_size, epochs = epochs, validation_split = 0.2, callbacks = [callback])

In [ ]:
plt.plot(np.arange(0, epochs), hist.history["loss"], label = "Train Loss")
plt.plot(np.arange(0, epochs), hist.history["val_loss"], label = "Validation Loss")
plt.legend()
plt.show()

In [ ]:
plt.plot(np.arange(0, epochs), hist.history["accuracy"], label = "Train Accuracy")
plt.plot(np.arange(0, epochs), hist.history["val_accuracy"], label = "Validation Accuracy")
plt.legend()
plt.show()

**Custom Data Set Training**

* Google Drive Connection

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

* Create Folder

In [ ]:
!mkdir dataset

* Unzip Custom Data Set

In [ ]:
!unzip "/content/drive/MyDrive/dataset/dataset.zip" -d "/content/dataset/"

In [ ]:
%cd "/content/dataset"

In [ ]:
!ls

* Check Data Set

In [ ]:
import cv2

In [ ]:
image = cv2.imread("/content/dataset/train/airplane/airplane_001.jpg", cv2.IMREAD_UNCHANGED)

In [ ]:
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [ ]:
plt.imshow(image)

* Get Image Shape

In [ ]:
input_shape = (64, 64, 3)

* Set Training Setting

In [ ]:
batch_size = 32
epochs = 5

* Data Augmentation

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(
                rescale = 1/255,
                rotation_range = 10,
                width_shift_range = 0.2,
                height_shift_range = 0.2,
                horizontal_flip = True,
                vertical_flip = True)

In [ ]:
valid_datagen = ImageDataGenerator(
                rescale = 1/255)

In [ ]:
test_datagen = ImageDataGenerator(
                rescale = 1/255)

* Load Custom Data Set

In [ ]:
train_loader = train_datagen.flow_from_directory(
                "/content/dataset/train", 
                target_size = (64, 64), 
                color_mode = "rgb", 
                classes = None,
                class_mode = "categorical", 
                batch_size = batch_size, 
                shuffle = True, 
                seed = None,
                interpolation = "bicubic"
                )

In [ ]:
valid_loader = valid_datagen.flow_from_directory(
                "/content/dataset/valid", 
                target_size = (64, 64), 
                color_mode = "rgb", 
                classes = None,
                class_mode = "categorical", 
                batch_size = batch_size, 
                shuffle = True, 
                seed = None,
                interpolation = "bicubic"
                )

In [ ]:
test_loader = test_datagen.flow_from_directory(
              "/content/dataset/test", 
              target_size = (64, 64), 
              color_mode = "rgb", 
              classes = None,
              class_mode = "categorical", 
              batch_size = batch_size, 
              shuffle = True, 
              seed = None,
              interpolation = "bicubic"
              )

* Get Number of Classes

In [ ]:
num_classes = train_loader.num_classes

In [ ]:
num_classes

* Assign CNN Model

In [ ]:
model = keras.Sequential()

In [ ]:
model.add(layers.Conv2D(32, (3, 3), padding = "same", activation = "relu", input_shape = input_shape))
model.add(layers.BatchNormalization())

model.add(layers.Conv2D(32, (3, 3), padding = "same", activation = "relu"))
model.add(layers.BatchNormalization())

model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), padding = "same", activation = "relu"))
model.add(layers.BatchNormalization())

model.add(layers.Conv2D(64, (3, 3), padding = "same", activation = "relu"))
model.add(layers.BatchNormalization())

model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), padding = "same", activation = "relu"))
model.add(layers.BatchNormalization())

model.add(layers.Conv2D(128, (3, 3), padding = "same", activation = "relu"))
model.add(layers.BatchNormalization())

model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(128, activation = "relu"))
model.add(layers.Dense(128, activation = "relu"))
model.add(layers.Dense(num_classes, "softmax"))

* Train Model

In [ ]:
model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])

In [ ]:
hist = model.fit(train_loader, epochs = epochs, callbacks = [callback], validation_data = valid_loader)

In [ ]:
plt.plot(np.arange(0, epochs), hist.history["loss"], label = "Train Loss")
plt.plot(np.arange(0, epochs), hist.history["val_loss"], label = "Validation Loss")
plt.legend()
plt.show()

In [ ]:
plt.plot(np.arange(0, epochs), hist.history["accuracy"], label = "Train Accuracy")
plt.plot(np.arange(0, epochs), hist.history["val_accuracy"], label = "Validation Accuracy")
plt.legend()
plt.show()